In [58]:
import spacy
import pandas as pd

In [59]:
nlp = spacy.load('en_core_web_lg')

In [60]:
def print_similarity(base_token,sentence):
    base_word = nlp(base_token)
    doc = nlp(sentence)
    list1 = []
    for token in doc:
        print(f'{token} <-> {base_word}',token.similarity(base_word))
        list1.append(token.similarity(base_word))
    print(max(list1))

In [61]:
text = "I love to eat sandwich and I will avuna it daily"
base_word = "car"
print_similarity(base_word,text)

I <-> car 0.18786868653430405
love <-> car 0.07109073504283019
to <-> car 0.12862019106815972
eat <-> car -0.014114078321992714
sandwich <-> car 0.1825516102197466
and <-> car 0.04201568263948912
I <-> car 0.18786868653430405
will <-> car 0.08284309647946639
avuna <-> car 0.0
it <-> car 0.248271946021776
daily <-> car 0.06606827923866984
0.248271946021776


C:\Users\hp\AppData\Local\Temp\ipykernel_5136\305989864.py:6: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  print(f'{token} <-> {base_word}',token.similarity(base_word))
C:\Users\hp\AppData\Local\Temp\ipykernel_5136\305989864.py:7: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  list1.append(token.similarity(base_word))


Fake news classification

In [62]:
import pandas

In [63]:
df= pd.read_csv('Fake_Real_Data.csv')
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [64]:
df.label = df.label.map({
    "Fake":0,
    "Real":1
})

In [65]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0
1,U.S. conservative leader optimistic of common ...,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",1
3,Court Forces Ohio To Allow Millions Of Illega...,0
4,Democrats say Trump agrees to work on immigrat...,1


In [66]:
def preprocessing(text):
    doc = nlp(text)
    list1 = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        list1.append(str(token))
    return " ".join(list1)

In [67]:
df.Text = df.Text.apply(preprocessing)

In [68]:
df['vector'] = df.Text.apply(lambda x: nlp(x).vector)

In [69]:
df.head()

,Text,label,vector
0,Trump Surrogate BRUTALLY Stabs Pathetic VIDE...,0,"[0.3797395, 1.3271066, -2.027777, 0.10855842, ..."
1,U.S. conservative leader optimistic common gro...,1,"[-0.73250884, 0.53248453, -1.4826572, 1.256643..."
2,Trump proposes U.S. tax overhaul stirs concern...,1,"[-1.1675037, -0.5304784, -2.4635859, 1.4895698..."
3,Court Forces Ohio Allow Millions Illegally P...,0,"[-1.8717813, 0.043430388, -1.4255608, 1.912064..."
4,Democrats Trump agrees work immigration bill w...,1,"[-0.9439954, 0.5562488, -2.0307415, 1.5689371,..."


In [70]:
from sklearn.model_selection import train_test_split as tst
X_train,X_test,y_train,y_test = tst(
    df.vector.values,
    df.label,
    random_state = 2022,
    test_size = 0.2
)

In [71]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [72]:
import numpy as np
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [73]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
X_train_2d = scaler.fit_transform(X_train_2d)
X_test_2d = scaler.fit_transform(X_test_2d)
clf.fit(X_train_2d,y_train)

MultinomialNB()

In [74]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1024
           1       0.95      0.98      0.96       956

    accuracy                           0.96      1980
   macro avg       0.96      0.96      0.96      1980
weighted avg       0.96      0.96      0.96      1980



In [75]:
from sklearn.neighbors import KNeighborsClassifier
clf1 = KNeighborsClassifier()
clf1.fit(X_train_2d,y_train)

KNeighborsClassifier()

In [76]:
y_pred = clf1.predict(X_test_2d)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1024
           1       0.98      0.99      0.99       956

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

